In [1]:
import os
import json
import cv2

with open("../Grounded-Segment-Anything/detect_results.json", "r") as f:
    data = json.load(f)

files = [x for x in os.listdir('ndh') if 'test' not in x]

panos = {}
for file in files:
    path = os.path.join('../transform_format/ndh', file)
    with open(path, 'r') as f:
        ndh_data = json.load(f)
        for ins in ndh_data:
            scan = ins['scan']
            end_panos = ins['end_panos']
            target = ins['target']
            instr_id = ins['instr_id']

            keys = []
            bboxes = []
            scores = []
            for end_pano in end_panos:
                for idx in range(36):
                    key = f'{scan}_{end_pano}_{target}_{idx}'
                    if key in data:
                        cands = data[key]
                        for jdx in range(len(cands)):
                            if data[key][jdx][1].split('(')[0] == target:
                                keys.append(key)
                                bboxes.append(data[key][jdx][0])
                                scores.append(float(data[key][jdx][1].split('(')[1].split(')')[0]))       

            if scores:
                max_idx = scores.index(max(scores))
                max_score = max(scores)
                assert max_score == scores[max_idx]
                max_key = keys[max_idx]
                view_index = max_key.split('_')[-1]
                view_pano = max_key.split('_')[1]
                panos[instr_id] = f"{scan}_{view_pano}_{view_index}"
            else:
                if int(instr_id) < 850:
                    panos[instr_id] = f"ac26ZMwG7aT_0bd07b7213b245f8a54ec4010f6ef1cc_3"
                elif int(instr_id) < 2500:
                    panos[instr_id] = f"aayBHfsNo7d_5278ace992664bbcb69d686a7be2c3b3_6"
                elif int(instr_id) < 3500:
                    panos[instr_id] = f"EDJbREhghzL_a748d2c29b664cf0b256534cee04eb43_4"
                else:
                    print(instr_id)

In [10]:
with open("../Inpainting/captions.json", "r") as f:
    captions = json.load(f)

from transformers import AutoTokenizer
def get_tokenizer():
    cfg_name = 'bert-base-uncased'
    tokenizer = AutoTokenizer.from_pretrained(cfg_name)
    return tokenizer

def encode_instr(tokenizer, instr):
    max_instr_len = 100
    instr_encoding = tokenizer.encode(instr, add_special_tokens=True)[:max_instr_len]
    return instr_encoding

def decode_instr(tokenizer, instr_encoding):
    instr = tokenizer.decode(instr_encoding, skip_special_tokens=True)
    return instr

files = [x for x in os.listdir('new_ndh') if 'test' not in x]
tok = get_tokenizer()
for file in files:
    path = os.path.join('../transform_format/new_ndh', file)
    with open(path, 'r') as f:
        cvdn_data = json.load(f)
        for i, ins in enumerate(cvdn_data):
            instr_id = ins['instr_id']
            instr_enc = ins['instr_encoding']
            instr = decode_instr(tok, instr_enc)
            caption = captions[panos[instr_id]]
            assert len(caption) == 1
            new_instr = instr + '. That room has ' + caption[0] + '.'
            new_instr_enc = encode_instr(tok, new_instr)
            cvdn_data[i]['instr_encoding'] = new_instr_enc
    
    os.makedirs('cvdn_long', exist_ok=True)
    with open(path.replace("new_ndh", "cvdn_long"), 'w') as f:
        json.dump(cvdn_data, f, indent=4)


In [4]:
with open("utsd_captions.json", "r") as f:
    captions = json.load(f)

from transformers import AutoTokenizer
def get_tokenizer():
    cfg_name = 'bert-base-uncased'
    tokenizer = AutoTokenizer.from_pretrained(cfg_name)
    return tokenizer

def encode_instr(tokenizer, instr):
    max_instr_len = 100
    instr_encoding = tokenizer.encode(instr, add_special_tokens=True)[:max_instr_len]
    return instr_encoding

def decode_instr(tokenizer, instr_encoding):
    instr = tokenizer.decode(instr_encoding, skip_special_tokens=True)
    return instr

files = [x for x in os.listdir('new_ndh') if 'test' not in x]
tok = get_tokenizer()
for file in files:
    path = os.path.join('../transform_format/new_ndh', file)
    with open(path, 'r') as f:
        cvdn_data = json.load(f)
        for i, ins in enumerate(cvdn_data):
            instr_id = str(ins['instr_id'])
            instr_enc = ins['instr_encoding']
            instr = decode_instr(tok, instr_enc)
            caption = captions[instr_id]
            assert len(caption) == 1
            new_instr = instr + '. That room has ' + caption[0] + '.'
            new_instr_enc = encode_instr(tok, new_instr)
            cvdn_data[i]['instr_encoding'] = new_instr_enc
    
    os.makedirs('utsd_obj_long', exist_ok=True)
    with open(path.replace("new_ndh", "utsd_obj_long"), 'w') as f:
        json.dump(cvdn_data, f, indent=4)